In [1]:
import import_ipynb
import threading
import time
import rsAruco as ra
import cv2
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
from IPython import display
# Dobot manipulation
import DobotDllType as dType
%matplotlib inline

importing Jupyter notebook from rsAruco.ipynb


In [2]:
# 创建新线程
thread1 = ra.cameraDetection(1, "rsArucoDetection")

def showImg():
    while True:
#         display.clear_output(wait=True)
        cv2.imwrite('./color.jpg', ra.color_image)
#         img=Image.open(r'./color.jpg')
#         plt.imshow(img)
    
thread2 = threading.Thread(target=showImg)

In [3]:
# start to find aruco code
if thread1.isAlive()==False:
    print("camera starts!")
    thread1.start()
    
time.sleep(2)
if thread2.isAlive()==False:
    print("showImg starts!")
    thread2.start()

camera starts!
Start streaming
showImg starts!


In [4]:
#dobot init
CON_STR = {
    dType.DobotConnect.DobotConnect_NoError:  "DobotConnect_NoError",
    dType.DobotConnect.DobotConnect_NotFound: "DobotConnect_NotFound",
    dType.DobotConnect.DobotConnect_Occupied: "DobotConnect_Occupied"}

#Load Dll
api = dType.load()


# Search 
# print(dType.SearchDobot(api))
#Connect Dobot
state = dType.ConnectDobot(api, "", 115200)[0]

print("Connect status:",CON_STR[state])

#set home postion
dType.SetHOMEParams(api,217,0,154,0)

# reset to home position
# dType.SetHOMECmd(api, temp = 0, isQueued = 1)

# Calibration points
default_cali_points = [[180,-120,135,0],[270,-120,135,0],
                       [180,120,135,0],[270,120,135,0],
                       [270,120,-5,0],[180,120,-5,0],
                       [180,-120,-5,0],[270,-120,-5,0]]
np_cali_points = np.array(default_cali_points)
arm_cord = np.column_stack((np_cali_points[:,0:3], np.ones(np_cali_points.shape[0]).T)).T

Connect status: DobotConnect_NoError


In [6]:
# 创建新线程
# thread1 = ra.cameraDetection(1, "rsArucoDetection", 1)

# start to find aruco code
# thread1.start()
#thread1.join() #阻塞

# clear queue
dType.SetQueuedCmdClear(api)
dType.SetQueuedCmdStartExec(api)

centers=np.ones(arm_cord.shape)

for ind,pt in enumerate(default_cali_points):
    print("Current points:", pt)
#     dType.SetQueuedCmdStopExec(api)
#     dType.SetQueuedCmdClear(api)
    queuedCmdIndex = dType.SetPTPCmd(api, 1, pt[0], pt[1], pt[2], pt[3], isQueued=0);
    while dType.GetQueuedCmdCurrentIndex(api) != queuedCmdIndex:
        time.sleep(1)
    time.sleep(2)
    centers[0:3,ind]=ra.center
    print(ra.center)
    

image_to_arm = np.dot(arm_cord, np.linalg.pinv(centers))
arm_to_image = np.linalg.pinv(image_to_arm)
dType.SetPTPCmd(api, 1, 217,0,154,0, isQueued=0);
dType.SetQueuedCmdStopExec(api);

print("Finished")
print("Image to arm transform:\n", image_to_arm)
print("Arm to Image transform:\n", arm_to_image)
print("Sanity Test:")

print("-------------------")
print("Image_to_Arm")
print("-------------------")
for ind, pt in enumerate(centers.T):
    print("Expected:", default_cali_points[ind][0:3])
    print("Result:", np.dot(image_to_arm, np.array(pt))[0:3])
    
print("-------------------")
print("Arm_to_Image")
print("-------------------")
for ind, pt in enumerate(default_cali_points):
    print("Expected:", centers.T[ind][0:3])
    pt[3]=1
    print("Result:", np.dot(arm_to_image, np.array(pt))[0:3])
# while 1:
#     time.sleep(1)
#     print("center:%f %f, depth:%f m" %(ra.center[0],ra.center[1],ra.center[2]))

Current points: [180, -120, 135, 1]
[-0.08698806166648865, -0.05967811867594719, 0.46800002455711365]
Current points: [270, -120, 135, 1]
[-0.0894770622253418, -0.012111172080039978, 0.39100003242492676]
Current points: [180, 120, 135, 1]
[0.13517193496227264, -0.059674132615327835, 0.4740000367164612]
Current points: [270, 120, 135, 1]
[0.13890340924263, -0.011433931067585945, 0.39500001072883606]
Current points: [270, 120, -5, 1]
[0.13773860037326813, 0.10304449498653412, 0.4690000116825104]
Current points: [180, 120, -5, 1]
[0.1345045119524002, 0.05438121408224106, 0.5480000376701355]
Current points: [180, -120, -5, 1]
[-0.08915271610021591, 0.05390601232647896, 0.5410000085830688]
Current points: [270, -120, -5, 1]
[-0.09075091034173965, 0.10191404074430466, 0.46300002932548523]
Finished
Image to arm transform:
 [[ 1.96126108e+01  5.29627209e+02 -8.24129260e+02  5.99464274e+02]
 [ 1.06299327e+03  4.15882530e+00  1.26712658e+01 -3.12660358e+01]
 [ 1.60079272e+01 -8.80002310e+02 -5.4

In [ ]:
# thread1.stop()
# thread1.isAlive()

In [ ]:
# threading.enumerate()

In [ ]:
# %whos